In [1]:
// Load some helper functions, e.g. to load values from settings.json
#!import config/Settings.cs 

// Import Semantic Kernel
#r "nuget: Azure.AI.OpenAI, 1.0.0-beta.12"
using Azure.AI.OpenAI;
var (model, azureEndpoint, apiKey) = Settings.LoadFromFile();


var endpoint = new Uri(azureEndpoint);
var credentials = new Azure.AzureKeyCredential(apiKey);
var openAIClient = new OpenAIClient(endpoint, credentials);

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.12

In [2]:
var system = 
""" 
Você é um assistente para classificar os sentimentos de um texto.
""";

In [3]:
var completionOptions = new ChatCompletionsOptions
{
    MaxTokens=400,
    Temperature=1f,
    FrequencyPenalty=0.0f,
    PresencePenalty=0.0f,
    NucleusSamplingFactor = 0.95f, // Top P
    DeploymentName = model
};

In [4]:
var functionDef = new FunctionDefinition()
        {
            Name = "reportSentiment",
            Description = "report the sentiment of the supplied text, and if it have sarcasm in it",
            Parameters = BinaryData.FromObjectAsJson(
            new
            {
                Type = "object",
                Properties = new
                {
                    Sentiment = new
                    {
                        Type = "string",
                        Description = "The sentiment of the text (positive, negative, neutral)",
                        num = new[] { "positive", "neutral", "negative" },
                    },
                    Sarcasm = new
                    {
                        Type = "boolean",
                        Description = "If the text contains sarcasm",
                    }
                },
                Required = new[] { "sentiment", "sarcasm" },
            },
            new JsonSerializerOptions() { PropertyNamingPolicy = JsonNamingPolicy.CamelCase }),
        };

In [5]:
completionOptions.Functions.Add(functionDef);
completionOptions.Messages.Add(new ChatRequestSystemMessage( system));

completionOptions

Azure.AI.OpenAI.ChatCompletionsOptions ChoiceCount <null> DeploymentName gpt-4-turbo FrequencyPenalty 0 MaxTokens 400 NucleusSamplingFactor 0.95 PresencePenalty 0 StopSequences [ ] IsUndefined True Count 0 IsReadOnly False (values) [ ] Temperature 1 TokenSelectionBiases (empty) Functions [ Azure.AI.OpenAI.FunctionDefinition ] IsUndefined False Count 1 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.FunctionDefinition Name reportSentiment Description report the sentiment of the supplied text, and if it have sarcasm in it Parameters {"type":"object","properties":{"sentiment":{"type":"string","description":"The sentiment of the text (positive, negative, neutral)","num":["positive","neutral","negative"]},"sarcasm":{"type":"boolean","description":"If the text contains sarcasm"}},"required":["sentiment","sarcasm"]} FunctionCall <null> AzureExtensionsOptions <null> ToolChoice <null> Messages [ Azure.AI.OpenAI.ChatRequestSystemMessage ] IsUndefined False Count 1 IsReadOnly False (values) index value 0 Azure.AI.OpenAI.ChatRequestSystemMessage Content Você é um assistente para classificar os sentimentos de um texto. Name <null> Role system User <null> Seed <null> ResponseFormat <null> Tools [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty)

In [6]:
completionOptions.Messages.Add(new ChatRequestUserMessage("O restaurante é muito bom, mas a comida é ruim."));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message


Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall Azure.AI.OpenAI.FunctionCall Name reportSentiment Arguments {"sentiment":"negative","sarcasm":false} AzureExtensionsContext <null>

In [7]:
completionOptions.Messages.Remove(completionOptions.Messages.Last());
completionOptions.Messages.Add(new ChatRequestUserMessage("O restaurante é tão chique que até as baratas usam gravata."));
ChatCompletions response = await openAIClient.GetChatCompletionsAsync(completionOptions);
response.Choices[0].Message


Azure.AI.OpenAI.ChatResponseMessage Role assistant Content <null> ToolCalls [ ] IsUndefined True Count 0 IsReadOnly False (values) (empty) FunctionCall Azure.AI.OpenAI.FunctionCall Name reportSentiment Arguments {"sentiment":"negative","sarcasm":true} AzureExtensionsContext <null>